In [9]:
import notebook_common
import jaconv
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [10]:
df_spoken = pd.read_csv(
    w_spoken_freq_nlb_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)

df_written = pd.read_csv(
    w_written_freq_nlt_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)


df_w_word_2400_without_guide_csv = pd.read_csv(
    w_word_2400_without_guide_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=0,
)
l("df_spoken")
df_spoken

----------------------------------------df_spoken----------------------------------------


,,spoken_freq
headword,hira,
の,の,57221425
に,に,35286930
を,を,33707106
は,は,31279365
が,が,30705135
...,...,...
ｋワット,きろわっと,1
右顧,うこ,1
右脳する,うのうする,1


In [11]:
l("df_written")
df_written

----------------------------------------df_written----------------------------------------


,,roman,written_freq
headword,hira,,
いる-非自立,いる,iru,968064
こと,こと,koto,701470
の,の,no,686025
する,する,suru,607405
ある,ある,aru,496737
...,...,...,...
所思,しょし,shoshi,1
襲用,しゅうよう,shuuyou,1
襲爵,しゅうしゃく,shuushaku,1


In [12]:
l("df_w_word_2400_without_guide_csv")
df_w_word_2400_without_guide_csv

----------------------------------------df_w_word_2400_without_guide_csv----------------------------------------


,word,chinese,property,meaning,new_chinese
no,,,,,
1,ああ,NaN,感,啊，嗯；啊呀，唉。,ああ
2,あい,愛,名・动３他,爱情；爱好。,愛
3,あいさつ,NaN,名・动３自,寒暄；致辞。,あいさつ
4,あいず,合図,名,信号。,合図
5,アイスクリーム,NaN,名,冰激凌；雪糕。,アイスクリーム
...,...,...,...,...,...
2415,わる,割る,动1他,"切,割;把一物分成几部分;除",割る
2416,わるい,悪い,形1,"坏;恶性;不对,错误",悪い
2417,われる,割れる,动2自,破碎;裂开,割れる


In [13]:
# 输出口语和书面语交集，索引是联合索引 拼写-发音
ids_df_spoken = set(df_spoken.index)
p(f"spoken freq index len:{len(ids_df_spoken)}")
ids_df_written = set(df_written.index)
p(f"written freq index len:{len(ids_df_written)}")
common_ids = ids_df_spoken & ids_df_written
p(f"common index len:{len(common_ids)}")

spoken freq index len:96948
written freq index len:84644
common index len:77273


In [14]:
# 原始书面语和口语词频合并
df_merged_raw = pd.concat([df_spoken, df_written], axis=1, join="outer")
to_csv_sig(df_merged_raw, w_raw_merged_freq_sumed_csv, True)
p("df_merged_raw--ok")
df_merged_raw

df_merged_raw--ok


,,spoken_freq,roman,written_freq
headword,hira,,,
の,の,57221425.0,no,686025.0
に,に,35286930.0,NaN,NaN
を,を,33707106.0,NaN,NaN
は,は,31279365.0,NaN,NaN
が,が,30705135.0,NaN,NaN
...,...,...,...,...
8ミリビデオカメラ,はちみりびでおかめら,NaN,hachimiribideokamera,1.0
鼻緒する,はなおする,NaN,hanaosuru,1.0
DJする,でぃーじぇいする,NaN,dhijeisuru,1.0


In [15]:
# 只以headword作为分组， 汇总spoken_freq和written_freq
result = (
    df_merged_raw.groupby("headword")[["spoken_freq", "written_freq"]]
    .sum()
    .reset_index()
)

result["cnt"] = result.groupby(["headword"]).transform("size")

headword_set = set(result["headword"])
p(len(headword_set))


to_csv_sig(result, "d:/tmp/merged_sum.csv")
p("ok")

102381
ok


In [16]:
# 作标准化处理 ，统计计算每4294,0640的词频, 以书面语词频作为基准
sum_spoken_freq = df_merged_raw["spoken_freq"].sum()
sum_written_freq = df_merged_raw["written_freq"].sum()
p(f"sum_spoken_freq:{sum_spoken_freq}")
p(f"sum_written_freq:{sum_written_freq}")
base_spoken_rate = sum_spoken_freq / sum_written_freq
# base_written_rate = sum_written_freq/10000000
p(f"base_spoken_rate:{base_spoken_rate}")
# p(base_written_rate)

sum_spoken_freq:949782694.0
sum_written_freq:42940640.0
base_spoken_rate:22.118503450344477


In [29]:
"""
2400词汇跟，合并后的书面语和口语的词频表进行左连接，用new_chinese进行连接

"""


df_spoken_written_freq_headword = (
    df_merged_raw.groupby("headword")[["spoken_freq", "written_freq"]]
    .sum()
    .reset_index()
)

# 对词频做标准化处理
df_spoken_written_freq_headword["spoken_freq"] = (
    df_spoken_written_freq_headword["spoken_freq"] / base_spoken_rate
)


# df_spoken_written_freq_headword
spoken_weight = 0.5
written_weight = 0.5
df_spoken_written_freq_headword["fused_freq"] = round(
    df_spoken_written_freq_headword["spoken_freq"] * spoken_weight
    + df_spoken_written_freq_headword["written_freq"] * written_weight
)


# 先用word进行连接
# t1 = pd.merge(df_w_word_2400_without_guide_csv,df_spoken_written_freq_headword,left_on='word',right_on='headword',how='left')
# t1_renamed = t1.rename(columns={'spoken_freq': 'word_spoken_freq', 'written_freq': 'word_written_freq','headword':'word_headword'})
# 再用chinese进行连接
# t2 = pd.merge(t1_renamed,df_spoken_written_freq_headword,left_on='chinese',right_on='headword',how='left')
# t2_renamed = t2.rename(columns={'spoken_freq': 'chinese_spoken_freq', 'written_freq': 'chinese_written_freq','headword':'chinese_headword'})

t1 = pd.merge(
    df_w_word_2400_without_guide_csv,
    df_spoken_written_freq_headword,
    left_on="new_chinese",
    right_on="headword",
    how="left",
)


to_csv_sig(t1, "d:/tmp/a.csv")
p("ok")

ok


In [26]:
df_merged = pd.concat([df_spoken, df_written], axis=1, join="outer")


def type_dection(spoken, written):
    if spoken > 0 and written > 0:
        return "both"
    elif np.isnan(spoken):
        return "written"
    elif np.isnan(written):
        return "spoken"
    else:
        return "empty"


df_merged["type"] = df_merged.apply(
    lambda x: type_dection(x["spoken_freq"], x["written_freq"]), axis=1
)

# 查看headword重复的次数
df_merged["headword_repetition"] = df_merged.groupby(["headword"]).transform("size")

# 查看reading重复的次数
df_merged["reading_repetition"] = df_merged.groupby(["reading"]).transform("size")

# 只对headword列进行统计
df_merged["gr_by_word_sum_s_freq"] = df_merged.groupby(["headword"])[
    "spoken_freq"
].transform("sum")
df_merged["gr_by_word_sum_w_freq"] = df_merged.groupby(["headword"])[
    "written_freq"
].transform("sum")

# df_merged.index
to_csv_sig(df_merged, w_merged_freq_sumed_csv, True)
p("w_merged_freq_sumed_csv--ok")

w_merged_freq_sumed_csv--ok


In [28]:
df_merged

,,spoken_freq,written_freq,type,headword_repetition,gr_by_word_sum_s_freq,gr_by_word_sum_w_freq
headword,reading,,,,,,
の,ノ,57221425.0,686025.0,both,1,57221425.0,686025.0
に,ニ,35286930.0,NaN,spoken,1,35286930.0,0.0
を,ヲ,33707106.0,NaN,spoken,1,33707106.0,0.0
は,ハ,31279365.0,NaN,spoken,1,31279365.0,0.0
が,ガ,30705135.0,NaN,spoken,1,30705135.0,0.0
...,...,...,...,...,...,...,...
8ミリビデオカメラ,ハチミリビデオカメラ,NaN,1.0,written,1,0.0,1.0
鼻緒する,ハナオスル,NaN,1.0,written,1,0.0,1.0
DJする,ディージェイスル,NaN,1.0,written,1,0.0,1.0


In [13]:
# word_in_headword  #找出2400价目词汇里有多少个在词频里
word2400_set = set(df_w_word_2400_without_guide_csv["new_chinese"])
p(len(word2400_set))

headword_freq_merge_set = set(df_merged.index.get_level_values(0))
p(len(headword_freq_merge_set))

reading_freq_merge_set = set(df_merged.index.get_level_values(1))
p(len(reading_freq_merge_set))

total_word = 0
effect = 0
word_in_headword = 0
word_in_reading = 0
for word in word2400_set:
    total_word += 1
    if word in headword_freq_merge_set:
        effect += 1
        word_in_headword += 1
        continue
    if word in reading_freq_merge_set:
        p(f"reading word:{word}")
        effect += 1
        word_in_reading += 1
        continue

p(f"effect:{effect}")
p(f"total_word:{total_word}")
p(f"word in headword:{word_in_headword}")
p(f"word in reading:{word_in_reading}")

p(effect / total_word)

2403
102381
80035
reading word:タイ
reading word:ティーシャツ
effect:2153
total_word:2403
word in headword:2151
word in reading:2
0.8959633791094466
